In [1]:
from premise import *
import bw2data, bw2io, bw2calc
#bw2data.projects.set_current("ecoinvent-3.12-cutoff")
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
#bw2data.projects.set_current("ei-3.11-bw25")

/opt/homebrew/Caskroom/miniforge/base/envs/premise/lib/python3.11/site-packages/scikits/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__('pkg_resources').declare_namespace(__name__)


12:20:58+0100 [warning  ] Can't import `SimaProBlockCSVImporter` - please install `bw2io` with `pip install bw2io[multifunctional]` or install `multifunctional` and `bw_simapro_csv` manually.


In [2]:
bw2data.databases

Databases dictionary with 12 object(s):
	biosphere3
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff
	ecoinvent-3.10.1-cutoff
	ei_cutoff_3.10.1_gcam_SSP2-Base_2050 2026-01-25
	ei_cutoff_3.10.1_image_SSP1-L_2025 2026-01-25
	ei_cutoff_3.10.1_remind_SSP1-NPi_2025 2026-01-25
	ei_cutoff_3.10.1_remind_SSP2-PkBudg650_2050 2026-02-04
	ei_cutoff_3.10.1_tiam-ucl_SSP2-RCP19_2050 2026-01-25
	test1
	test2
	test3

In [3]:
clear_inventory_cache()

Inventory cache cleared!


In [4]:
scenarios = [
    #{"model": "remind", "pathway": "SSP1-NPi", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-PkBudg650", "year": 2025,},
    #{"model": "remind", "pathway": "SSP1-PkBudg1000", "year": 2025,},
    
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2025,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2025,},
    {"model": "remind", "pathway": "SSP2-PkBudg650", "year": 2050,},
    #{"model": "remind", "pathway": "SSP2-PkBudg1000", "year": 2025,},
    
    #{"model": "remind", "pathway": "SSP3-rollBack", "year": 2025,},

    #{"model": "remind-eu", "pathway": "SSP2-NPi", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-NDC", "year": 2025,},
    #{"model": "remind-eu", "pathway": "SSP2-PkBudg650", "year": 2050,},
    #{"model": "remind-eu", "pathway": "SSP2-PkBudg1000", "year": 2025,},

    #{"model": "image", "pathway": "SSP1-L", "year": 2025,},
    #{"model": "image", "pathway": "SSP1-Ma", "year": 2025,},
    #{"model": "image", "pathway": "SSP1-VLLO", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-L", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-M", "year": 2025,},
    #{"model": "image", "pathway": "SSP2-VLHO", "year": 2025,},
    #{"model": "image", "pathway": "SSP5-H", "year": 2025,},
    #{"model": "image", "pathway": "SSP3-H", "year": 2025,},

    #{"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2050,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP26", "year": 2025,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP45", "year": 2025,},

    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-NPi", "year": 2050,},

    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-NDC", "year": 2050,},

    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2030,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2035,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2040,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2045,},
    #{"model": "remind", "pathway": "SSP2-rollBack", "year": 2050,},

    #{"model": "gcam", "pathway": "SSP2-Base", "year": 2050,},
    #{"model": "gcam", "pathway": "SSP2-RCP26", "year": 2050,},
    #{"model": "tiam-ucl", "pathway": "SSP2-RCP19", "year": 2050}
]


ndb = NewDatabase(
        scenarios = scenarios,
        source_db="ecoinvent-3.10.1-cutoff",
        source_version="3.10.1",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        biosphere_name="biosphere3",
)


premise v.(2, 3, 6)
+------------------------------------------------------------------+
| Warning                                                          |
+------------------------------------------------------------------+
| Because some of the scenarios can yield LCI databases            |
| containing net negative emission technologies (NET),             |
| it is advised to account for biogenic CO2 flows when calculating |
| Global Warming potential indicators.                             |
| `premise_gwp` provides characterization factors for such flows.  |
| It also provides factors for hydrogen emissions to air.          |
|                                                                  |
| Within your Brightway project:                                   |
| from premise_gwp import add_premise_gwp                          |
| add_premise_gwp()                                                |
+------------------------------------------------------------------+
+-------------

In [5]:
ndb.update()

Processing scenarios for all sectors: 100%|█| 1/1 [01:40<00:00, 100.26

Done!



In [6]:
ndb.write_db_to_brightway()

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
Database ei_cutoff_3.10.1_remind_SSP2-PkBudg650_2050 2026-02-04 already exists: it will be overwritten.
12:24:30+0100 [info     ] Vacuuming database            
12:25:49+0100 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████| 37284/37284 [00:57<00:00, 649.32it/s]

12:26:52+0100 [info     ] Vacuuming database            


Created database: ei_cutoff_3.10.1_remind_SSP2-PkBudg650_2050 2026-02-04
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev/export/change reports/.


In [ ]:
ndb.scenarios[0]["iam data"].steel_technology_mix.sel(region="AFR", year=2050)

In [ ]:
ndb.scenarios[0]["iam data"].production_volumes.sel(region="AFR", variables=[
    'steel - primary - BF/BOF', 'steel - primary - BF/BOF CCS',
       'steel - primary - DRI', 'steel - primary - DRI CCS',
       'steel - primary - H-DRI', 'steel - secondary'
], year=2050).sum(dim="variables")

In [ ]:
ndb.scenarios[0]["iam data"].production_volumes.sel(region="AFR", variables=[
    'steel - primary - BF/BOF', 'steel - primary - BF/BOF CCS',
       'steel - primary - DRI', 'steel - primary - DRI CCS',
       'steel - primary - H-DRI', 'steel - secondary'
], year=2050)

In [5]:
ndb.update("steel")

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
16:18:30+0100 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|████████████████████████████████████| 37347/37347 [01:05<00:00, 570.84it/s]


16:19:49+0100 [info     ] Vacuuming database            
Created database: test carina 2
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev/export/change reports/.


In [5]:
ndb.write_db_to_brightway("test david 9")

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.
Database test david 1 already exists: it will be overwritten.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:21


Title: Writing activities to SQLite3 database:
  Started: 11/15/2025 19:50:03
  Finished: 11/15/2025 19:50:24
  Total time elapsed: 00:00:21
  CPU %: 89.00
  Memory %: 10.03
Created database: test david 1
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev/export/change reports/.


In [3]:
from premise import *
import bw2data, bw2io, bw2calc
#bw2data.projects.set_current("bw25_ei310")
bw2data.projects.set_current("ecoinvent-3.11-cutoff-bw25")

Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.


In [6]:
scenarios = [
    {"model": "remind", "pathway": "SSP1-NPi",},
]


ndb = PathwaysDataPackage(
        scenarios = scenarios,
        years=[2050,],
        source_db="ecoinvent-3.11-cutoff",
        source_version="3.11",
        key='tUePmX_S5B8ieZkkM7WUU2CnO8SmShwmAeWK9x2rTFo=',
        biosphere_name="ecoinvent-3.11-biosphere",
        use_absolute_efficiency=True,
)

ndb.create_datapackage()

Processing scenarios for all sectors:   0%|     | 0/2 [00:00<?, ?it/s]

No buildings heat scenario data available -- skipping
No industrial heat scenario data available -- skipping
No DAC energy mix data available -- skipping
No EWR energy mix data available -- skipping
No car fleet scenario data available -- skipping
No two-wheeler fleet scenario data available -- skipping
No truck fleet scenario data available -- skipping
No ship fleet scenario data available -- skipping
No bus fleet scenario data available -- skipping
No train fleet scenario data available -- skipping
No final energy scenario data available -- skipping


Processing scenarios for all sectors:  50%|▌| 1/2 [01:53<01:53, 113.65

No buildings heat scenario data available -- skipping
No industrial heat scenario data available -- skipping
No DAC energy mix data available -- skipping
No EWR energy mix data available -- skipping
No car fleet scenario data available -- skipping
No two-wheeler fleet scenario data available -- skipping
No truck fleet scenario data available -- skipping
No ship fleet scenario data available -- skipping
No bus fleet scenario data available -- skipping
No train fleet scenario data available -- skipping
No final energy scenario data available -- skipping


Processing scenarios for all sectors: 100%|█| 2/2 [03:47<00:00, 113.88

Done!



In [7]:

for k, v in ndb.datapackage.scenarios[0]["mapping"]["capacity"].items():
    for ds in v:
        print(k, "|", ds["name"], "|", ds["unit"])

Write new database(s) to Brightway.
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:36


Title: Writing activities to SQLite3 database:
  Started: 10/22/2025 10:23:46
  Finished: 10/22/2025 10:24:23
  Total time elapsed: 00:00:36
  CPU %: 88.40
  Memory %: 13.23
Created database: ei_cutoff_3.11_gcam_SSP2-Base_2050 2025-10-22
Running all checks...
Minor anomalies found: check the change report.


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:37


Title: Writing activities to SQLite3 database:
  Started: 10/22/2025 10:39:06
  Finished: 10/22/2025 10:39:44
  Total time elapsed: 00:00:37
  CPU %: 87.60
  Memory %: 13.53
Created database: ei_cutoff_3.11_gcam_SSP2-RCP26_2050 2025-10-22
Generate scenario report.
Report saved under /Users/romain/GitHub/premise/dev/export/scenario_report.
Generate change report.
Report saved under /Users/romain/GitHub/premise/dev/export/change reports/.


In [6]:
list(ndb.datapackage.scenarios[0]["mapping"].keys())

market for steel, unalloyed SSA 126.5147048
market for steel, unalloyed NEN 3.9033624000000002
market for steel, unalloyed DEU 30.9445126
market for steel, unalloyed CHA 627.594136
market for steel, unalloyed NES 33.7435476
market for steel, unalloyed ESC 28.518298299999998
market for steel, unalloyed USA 125.8700527
market for steel, unalloyed ECS 3.1706321
market for steel, unalloyed FRA 12.4022449
market for steel, unalloyed CAZ 20.2429662
market for steel, unalloyed IND 245.47239739999998
market for steel, unalloyed REF 44.1181361
market for steel, unalloyed JPN 77.40011770000001
market for steel, unalloyed ESW 16.153458699999998
market for steel, unalloyed MEA 120.0108343
market for steel, unalloyed ECE 12.3216091
market for steel, unalloyed UKI 7.385962899999999
market for steel, unalloyed EWN 25.3922545
market for steel, unalloyed LAM 101.8951347
market for steel, unalloyed ENC 12.5858469
market for steel, unalloyed OAS 168.2978058
market for steel, unalloyed GLO 0.0
steel produ

In [6]:
list(ndb.datapackage.scenarios[0]["mapping"]["capacity"].keys())

'market for erbium oxide' (kilogram, GLO, None)
'market for titanium' (kilogram, GLO, None)
'market for tungsten concentrate' (kilogram, GLO, None)
'market for copper, cathode' (kilogram, GLO, None)
'market for sulfur' (kilogram, GLO, None)
'market for cobalt' (kilogram, GLO, None)
'market for chromite ore concentrate' (kilogram, GLO, None)
'market for ilmenite, 54% titanium dioxide' (kilogram, GLO, None)
'market for aluminium, primary, liquid' (kilogram, GLO, None)
'market for boron carbide' (kilogram, GLO, None)
'market for ytterbium oxide' (kilogram, GLO, None)
'market for uranium ore, as U' (kilogram, GLO, None)
'market for beryllium' (kilogram, GLO, None)
'market for silicon, metallurgical grade' (kilogram, GLO, None)
'market for gold' (kilogram, GLO, None)
'market for sodium borates' (kilogram, GLO, None)
'market for copper, anode' (kilogram, GLO, None)
'market for spodumene' (kilogram, GLO, None)
'market for praseodymium oxide' (kilogram, GLO, None)
'market for lead concentrate'

In [7]:
ei = bw2data.Database("ecoinvent-3.11-cutoff")

In [26]:
bw2data.databases

[('ISIC rev.4 ecoinvent', '4220:Construction of utility projects'), ('CPC', '53262: Power plants')]


In [ ]:
len(ei)

In [ ]:
for ds in ei:
    if "boiler" in ds["name"] and ds["unit"] == "unit":
        print(ds["classifications"])

In [ ]:
from premise.utils import load_database

ndb.scenarios[0] = load_database(ndb.scenarios[0], ndb.database)

In [ ]:
db = ndb.scenarios[0]["database"]

In [ ]:
ei = bw2data.Database("ecoinvent-3.11-cutoff")

In [ ]:
for ds in ei:
    if "wind power plant construction, 2MW, offshore, fixed parts" in ds["name"]:
        print(ds["classifications"])

In [ ]:
for ds in db:
    if "Comment" in ds:
        print(ds["name"])
    for e in ds["exchanges"]:
        if "Comment" in e:
            print(ds["name"])

In [ ]:
ndb.write_db_to_brightway("test david")

In [ ]:
ndb.update()

In [ ]:
ndb.write_db_to_brightway()

In [ ]:
db = bw2data.Database("ei_cutoff_3.11_remind-eu_SSP2-PkBudg650_2050 2025-09-10")
for ds in db:
    if ds["name"].startswith("market for steel, unalloyed"):
        for exc in ds.production():
            if "production volume" in exc:
                print(ds["name"], ds["location"], exc["production volume"])

for ds in db:
    if ds["name"].startswith("steel production, "):
        for exc in ds.production():
            if "production volume" in exc:
                print(ds["name"], ds["location"], exc["production volume"])

In [ ]:
import bw2data
bw2data.projects.set_current("ecoinvent-3.11-cutoff")
db = bw2data.Database("test metals 3")

markets_list = []
for ds in db:
    if ds["location"] == "World":
        markets_list.append(ds["name"])

dbs = [
    "ecoinvent-3.11-cutoff",
    "test metals 3"
]
fus = []
for d in dbs:
    for ds in bw2data.Database(d):
        if ds["name"] in markets_list and ds["location"] in ("GLO", "World"):
            if d == "test metals 3":
                if ds["location"] == "World":
                    fus.append(ds)
            else:
                fus.append(ds)

selected_methods = [m for m in bw2data.methods if "ef v3.1 no" in str(m).lower()]

import bw2calc
import numpy as np
from bw2calc.errors import OutsideTechnosphere


def get_lcia_scores(products, categories, results):
    lca = bw2calc.LCA({products[0]: 1}, categories[0])
    lca.lci()
    lca.lcia()
    method_matrices = [lca.characterization_matrix.copy()]
    for other_method in categories[1:]:
        # Only build each characterization matrix once instead of once per product
        lca.switch_method(other_method)
        method_matrices.append(lca.characterization_matrix.copy())
    for i, product in enumerate(products):
        print(product)
        try:
            lca.redo_lci({product: 1})
        except OutsideTechnosphere:
            lca = bw2calc.LCA({product: 1}, categories[0])
            lca.lci()
            lca.lcia()

            method_matrices = [lca.characterization_matrix.copy()]
            for other_method in categories[1:]:
                # Only build each characterization matrix once instead of once per product
                lca.switch_method(other_method)
                method_matrices.append(lca.characterization_matrix.copy())
        for j, characterization_matrix in enumerate(method_matrices):
            results[i, j] = (characterization_matrix * lca.inventory).sum()
    return results

results = np.zeros((len(fus), len(selected_methods)))
results = get_lcia_scores(fus, selected_methods, results)


import pandas as pd
df = pd.DataFrame(columns=[e[1] for e in selected_methods], index=[x["name"] for x in fus], data=results)

df["database"] = [x["database"] for x in fus]
df["location"] = [x["location"] for x in fus]

# Step 1: Copy dataframe
df_rel = df.copy()

# Identify columns with impact scores (exclude identifiers)
id_cols = [ "database", "location"]
impact_cols = [c for c in df.columns if c not in id_cols]

# Step 2: Normalize relative to ecoinvent-3.11-cutoff (index 100)
df_rel_list = []
for metal, group in df.groupby(df.index):
    ref = group[group["database"] == "ecoinvent-3.11-cutoff"][impact_cols]
    if ref.empty:
        continue
    ref_values = ref.iloc[0]
    # Compute relative values
    temp = group.copy()
    for col in impact_cols:
        temp[col] = (temp[col] / ref_values[col]) * 100 if ref_values[col] != 0 else None
    df_rel_list.append(temp)

df_rel = pd.concat(df_rel_list, ignore_index=False)

# Step 3: Save to new Excel file
out_path = "comparison_metals_relative.xlsx"
df_rel.to_excel(out_path, index=True)